In [456]:
# Imports
import os
import requests
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import hvplot.pandas
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import quantstats as qs
%matplotlib inline

In [457]:
# Creating Charts from a quantstats library
# From library extend pandas functionality with metrics, etc.
qs.extend_pandas()

# fetch the daily returns for a stock
stock = qs.utils.download_returns('BTC')

# show sharpe ratio
qs.stats.sharpe(stock)

# or using extend_pandas()
stock.sharpe()

# Plot 
qs.plots.snapshot(stock, title='BTC Performance')

In [ ]:
# Create 6 charts from the quantstats
qs.plots.yearly_returns(stock)
qs.plots.monthly_heatmap(stock)
qs.plots.daily_returns(stock)
qs.plots.rolling_sharpe(stock)
qs.plots.drawdown(stock)
qs.plots.log_returns(stock)


In [ ]:
# Load .env enviroment variables
load_dotenv()

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
# Pulling Crypto Information and store in datframe
# Format current date as ISO format
today = pd.Timestamp("2021-1-1", tz="America/New_York").isoformat()

# Set the tickers BTCUSD (bitcoin), ETHUSD (Ethereum), and TRXUSD (Tron)
tickers = ["BTCUSD", "ETHUSD","TRXUSD"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for crypto

df_crypto = alpaca.get_crypto_bars(
    tickers,
    timeframe,
    start = today
).df


In [ ]:
# Reorganize the crypto DataFrame and clean up data
# 
BTC = df_crypto[df_crypto['symbol']=='BTCUSD'].drop('symbol', axis=1)
BTC['BTC Return'] = BTC['close'].pct_change()
BTC = BTC.drop(columns= ['exchange','open','volume', 'trade_count', 'vwap'])

ETH = df_crypto[df_crypto['symbol']=='ETHUSD'].drop('symbol', axis=1)
ETH['ETH Return'] = ETH['close'].pct_change()
ETH = ETH.drop(columns= ['exchange','open','volume', 'trade_count', 'vwap'])

TRX = df_crypto[df_crypto['symbol']=='TRXUSD'].drop('symbol', axis=1)
TRX['TRX Return'] = TRX['close'].pct_change()
TRX = TRX.drop(columns= ['exchange','open','volume', 'trade_count', 'vwap'])

# Concatenate the ticker DataFrames
df_crypto = pd.concat([BTC, ETH, TRX],axis=1, keys=['BTC','ETH', 'TRX']).dropna()

df_crypto.head()

In [ ]:
# Pulling Stock Information and store in datframe
# Format current date as ISO format
today = pd.Timestamp("2021-1-24", tz="America/New_York").isoformat()

# Set the stock_tickers to TSLA , GME, and SPY
stock_tickers = ["TSLA", "GME","SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and GOOGL
# (use a limit=1000 parameter to call the most recent 1000 days of data)
df_stock = alpaca.get_bars(
    stock_tickers,
    timeframe,
    start = today
).df

In [ ]:
# Reorganize the stocd DataFrame
# Separate ticker data
TSLA = df_stock[df_stock['symbol']=='TSLA'].drop('symbol', axis=1)
TSLA['TSLA Return'] = TSLA['close'].pct_change().dropna()
TSLA = TSLA.drop(columns= ['open','volume', 'trade_count', 'vwap'])

GME = df_stock [df_stock['symbol']=='GME'].drop('symbol', axis=1)
GME['GME Return'] = GME['close'].pct_change().dropna()
GME = GME.drop(columns= ['open','volume', 'trade_count', 'vwap'])

SPY = df_stock [df_stock['symbol']=='SPY'].drop('symbol', axis=1)
SPY['SPY Return'] = SPY['close'].pct_change().dropna()
SPY = SPY.drop(columns= ['open','volume', 'trade_count', 'vwap'])

# Concatenate the ticker DataFrames
df_stock = pd.concat([TSLA, GME, SPY],axis=1, keys=['TSLA','GME', 'SPY']).dropna()
df_stock.head()

In [ ]:
# Drawdown GME chart

# Trailing 252 trading day window
window = 252

# Calculate the max drawdown in the past window days for each day in the series.
# Use min_periods=1 if you want to let the first 252 days data have an expanding window
roll_max = df_stock['GME']['close'].rolling(window, min_periods=1).max()
daily_drawdown = df_stock['GME']['close']/roll_max - 1.0

# Next we calculate the minimum (negative) daily drawdown in that window.
# Again, use min_periods=1 if you want to allow the expanding window
max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()

# Plot the results
d = daily_drawdown.hvplot(legend = False, hover_color = "yellow")
md = max_daily_drawdown.hvplot(title = "GME Max Daily Drawdown", legend= False, hover_color = "green")

#overlay both daily drawdown and max daily drawdown
d*md

In [ ]:
#Combine crypto and stock returns
df_crypto_return =  pd.concat([df_crypto['BTC']['BTC Return'], df_crypto['ETH']['ETH Return'],df_crypto['TRX']['TRX Return']], axis=1)
df_stock_return = pd.concat([df_stock['TSLA']['TSLA Return'], df_stock['GME']['GME Return'], df_stock['SPY']['SPY Return']], axis =1)

# Calculate the log return
df_log_return = np.log(1 + df_crypto_return)
df_log_return_stock = np.log(1 + df_stock_return)

# overlay the two charts
crypto_log = df_log_return.hvplot(title = "Log Daily Return")
stock_log = df_log_return_stock.hvplot()
stock_log * crypto_log

In [ ]:
# Display Crypto and Stock
display(df_crypto_return.tail())
display(df_stock_return.tail())


In [ ]:
# Plot the daily returns
c = df_crypto_return.hvplot(kind="line", title = "Daily Returns")
s = df_stock_return.hvplot(kind="line")
c * s

In [ ]:
#Calculate cumulative returns for crypto and stocks
df_cumulative_returns = (1 + df_crypto_return).cumprod()
df_cumulative_stock_returns = (1 + df_stock_return).cumprod()


# Plot the chart
c = df_cumulative_returns.hvplot(title = "Cumulative Returns")
s = df_cumulative_stock_returns.hvplot()
c*s

In [ ]:
#Plot box chart
c = df_crypto_return.hvplot.box( title = "Daily Return Box", legend = False)
s = df_stock_return.hvplot.box(legend = False)
c*s

In [ ]:
#calculate STDs
all_stds = df_crypto_return.std()
stock_stds = df_stock_return.std()
all_stds = pd.concat([all_stds, stock_stds], axis=0)
all_stds.hvplot.bar(legend = False, title = "STDs", hover_fill_color= "purple")

In [ ]:
#Volatity
volatility = all_stds * np.sqrt(365)
volatility.hvplot.line(title = "Volatility")

In [ ]:
#Plot rolling window
rolling_21 = df_crypto_return.rolling(window=21).std().plot(title = "Crypto Rolling 21 Days Return")

In [ ]:
#Plot rolling
rolling_21 = df_stock_return.rolling(window=21).std().plot(title = "Stock Rolling 21 Days Return")

In [ ]:
#Calculate correlation and plot heatmap

#Aggregate the timestamps to 1D for all the crypto
b = df_crypto['BTC'].resample('1D')['BTC Return'].agg(['first'])
e = df_crypto['ETH'].resample('1D')['ETH Return'].agg(['first'])
t = df_crypto['TRX'].resample('1D')['TRX Return'].agg(['first'])

# Rename all of the first columns
b.rename(columns= {'first':'BTC Return'}, inplace = True)
e.rename(columns= {'first':'ETH Return'}, inplace = True)
t.rename(columns= {'first':'TRX Return'}, inplace = True)

# Combine the BTC/ETH/TRX Return and reset index to date
df_bet = pd.concat([b, e, t], axis=1)
df_bet.index=df_bet.index.date

# Copy df_stock_return and reset index to date
df0 = df_stock_return
df0.index = df_stock_return.index.date

# Combine both Crypto and Stock Returns
df_cor = pd.concat([df0, df0], axis=1).dropna()

# Plot correlations
display(sns.heatmap(df_cor.corr(), cmap ="YlGnBu", linewidths = 0.1))


In [ ]:
#Calculate GME Rolling Covariance vs SP500
covariance = df_stock_return['GME Return'].cov(df_stock_return['SPY Return'])

# Calculate variance and beta
variance = df_stock_return['SPY Return'].var()
GME_beta = covariance / variance

# plot rolling covariance vs. SP500
rolling_covariance = df_stock_return['GME Return'].rolling(window=21).cov(df_stock_return['SPY Return'])
rolling_covariance.plot(title="GME 21-Day Rolling Covariance vs SP500")

In [ ]:
# Calculate the covariance for TRX and ETH
covariance2 = df_crypto_return['TRX Return'].cov(df_crypto_return['ETH Return'])

variance2 = df_crypto_return['ETH Return'].var()
TRX_beta2 = covariance2 / variance2
rolling_covariance2 = df_crypto_return['TRX Return'].rolling(window=21)

rolling_covariance2 = df_crypto_return['TRX Return'].rolling(window=21).cov(df_crypto_return['ETH Return'])

#Plot the chart
rolling_covariance2.plot(title="TRX 21-Day Rolling Covariance vs ETH")

In [ ]:
#Calculate ewm and plot it
ewm = df_crypto_return.ewm(halflife=21).std().dropna()
ewm.plot()

In [ ]:
# Create rolling sharpe function
def my_rolling_sharpe(y):
    return np.sqrt(126) * (y.mean() / y.std()) # 21 days per month X 6 months = 126

# Create new dataframe
df_c_return = pd.DataFrame()

# Call rolling sharpe function
df_c_return['Rolling Sharpe'] = df_crypto_return['BTC Return'].rolling('180d').apply(my_rolling_sharpe)

# Plot chart
df_c_return.plot(y=['Rolling Sharpe'], linewidth=0.5, title = "BTC Rolling Sharpe - 6 Months")

In [ ]:
# create rolling sharpe function
def my_rolling_sharpe(y):
    return np.sqrt(126) * (y.mean() / y.std()) # 21 days per month X 6 months = 126

# create new dataframe
df_e_return = pd.DataFrame()

# Call rolling sharpe function
df_e_return['Rolling Sharpe'] = df_crypto_return['ETH Return'].rolling('180d').apply(my_rolling_sharpe)

# Plot chart
df_e_return.plot(y=['Rolling Sharpe'], linewidth=0.5, title = "ETH Rolling Sharpe - 6 Months")

In [ ]:
# Calculate the Sharpe ratios
sharpe_crypto_ratios = (df_crypto_return.mean() * 252) / (df_crypto_return.std() * np.sqrt(252))
sharpe_stock_ratios = (df_stock_return.mean() * 252) / (df_stock_return.std() * np.sqrt(252))


# Plot Sharpe ratios for crypto and stocks
c = sharpe_crypto_ratios.hvplot.bar(title = "Sharpe Ratios", hover_fill_color="orange", legend = False)
s = sharpe_stock_ratios.hvplot.bar(hover_color= "orange", legend= False)
c*s

In [ ]:
# Monthly Returns for crypto
crypto_monthly_returns = df_crypto['ETH'].resample('1M')['close'].agg(['first']).pct_change().dropna()

# Convert to Dataframe
crypto_monthly_returns = pd.DataFrame(crypto_monthly_returns)
crypto_monthly_returns.index = crypto_monthly_returns.index.date

# Create year and month columns
crypto_monthly_returns['year'] = pd.DatetimeIndex(crypto_monthly_returns.index).year
crypto_monthly_returns['month'] = pd.DatetimeIndex(crypto_monthly_returns.index).month_name().astype(str)

# Convert Dataframe to a pivot table
df1 = crypto_monthly_returns.pivot("year", "month", "first")
df1= df1.fillna(0.00)

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(df1, annot=True, fmt=".1g", linewidths=.5, ax=ax, cmap='RdYlGn')
ax.set_title('ETH Monthly Returns (% Change)')


In [ ]:
# Daily and Yearly Returns for Stocks
data_daily_returns = df_stock['SPY']['close'].pct_change()
data_yearly_returns =df_stock['TSLA']['close'].resample('A').ffill().pct_change().dropna()

# Monthly Returns for stocks
data_monthly_returns = df_stock['GME']['close'].resample('M').ffill().pct_change().dropna()


# Convert to Dataframe
data_monthly_returns = pd.DataFrame(data_monthly_returns)
data_monthly_returns.index = data_monthly_returns.index.date

# Create year and month columns
data_monthly_returns['year'] = pd.DatetimeIndex(data_monthly_returns.index).year
data_monthly_returns['month'] = pd.DatetimeIndex(data_monthly_returns.index).month_name().astype(str)

# Convert Dataframe to a pivot table
df1 = data_monthly_returns.pivot("year", "month", "close")
df1= df1.fillna(0.00)

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(df1, annot=True, fmt=".1g", linewidths=.5, ax=ax, cmap='RdYlGn')
ax.set_title('GME Monthly Returns (% Change)')



In [ ]:
# Plot the yearly % Change

# Yearly Returns for crypto
crypto_yearly_returns =df_crypto['TRX'].resample('AS-NOV')['close'].agg(['first']).pct_change().dropna()

crypto_yearly_returns.hvplot.bar(title = "TRX Yearly % Change as of November", ylabel= "% Change", xlabel= "Year", hover_fill_color= "green")
